In [1]:
from Bio import SeqIO
import pandas as pd
import re

In [26]:
x = r'\(([^)]+)\)'

In [2]:
desc = []
seq = []
c=0
for sr in SeqIO.parse('GCF_000001405.39_GRCh38.p13_rna.fna', 'fasta'):
    print(sr)
    print(sr.seq)
    break

ID: NM_000014.6
Name: NM_000014.6
Description: NM_000014.6 Homo sapiens alpha-2-macroglobulin (A2M), transcript variant 1, mRNA
Number of features: 0
Seq('GGGACCAGATGGATTGTAGGGAGTAGGGTACAATACAGTCTGTTCTCCTCCAGC...GTC', SingleLetterAlphabet())
GGGACCAGATGGATTGTAGGGAGTAGGGTACAATACAGTCTGTTCTCCTCCAGCTCCTTCTTTCTGCAACATGGGGAAGAACAAACTCCTTCATCCAAGTCTGGTTCTTCTCCTCTTGGTCCTCCTGCCCACAGACGCCTCAGTCTCTGGAAAACCGCAGTATATGGTTCTGGTCCCCTCCCTGCTCCACACTGAGACCACTGAGAAGGGCTGTGTCCTTCTGAGCTACCTGAATGAGACAGTGACTGTAAGTGCTTCCTTGGAGTCTGTCAGGGGAAACAGGAGCCTCTTCACTGACCTGGAGGCGGAGAATGACGTACTCCACTGTGTCGCCTTCGCTGTCCCAAAGTCTTCATCCAATGAGGAGGTAATGTTCCTCACTGTCCAAGTGAAAGGACCAACCCAAGAATTTAAGAAGCGGACCACAGTGATGGTTAAGAACGAGGACAGTCTGGTCTTTGTCCAGACAGACAAATCAATCTACAAACCAGGGCAGACAGTGAAATTTCGTGTTGTCTCCATGGATGAAAACTTTCACCCCCTGAATGAGTTGATTCCACTAGTATACATTCAGGATCCCAAAGGAAATCGCATCGCACAATGGCAGAGTTTCCAGTTAGAGGGTGGCCTCAAGCAATTTTCTTTTCCCCTCTCATCAGAGCCCTTCCAGGGCTCCTACAAGGTGGTGGTACAGAAGAAATCAGGTGGAAGGACAGAGCACCCTTTCACCGTGGAGGAATTTGTTCTTCCCAAGTTTG

In [34]:
acr = []
for i in desc:
    match = re.search(x, i)
    acr.append(match.group(1))

In [35]:
df = pd.DataFrame({'Desc' : desc, 'Acr' : acr, 'Seq' : seq})

In [39]:
df['Len'] = 0

for i,r in df.iterrows():
    df.loc[i,'Len'] = len(r['Seq'])

In [40]:
#df.to_csv('hg38 des acr len.csv', index = False)

In [2]:
df = pd.read_csv('hg38 des acr len.csv')

In [4]:
#count PREDICTED

c = 0
for d in df['Desc'].to_list():
    if 'PREDICTED' in d:
        c+=1
print(c)

#drop PREDICTED

df['Pred'] = 0
for i,r in df.iterrows():
    if 'PREDICTED' in r['Desc']:
        df.loc[i,'Pred'] = 1

86694


In [7]:
df = df.sort_values(by = 'Len', ascending = False).reset_index(drop = True)
df = df.loc[df['Pred'] == 0]
df = df.drop_duplicates('Acr').reset_index(drop = True)

In [8]:
df = df.loc[df['Len'] > 2000]
df = df.loc[df['Len'] < 35000].reset_index(drop = True)

In [118]:
#df.to_csv('hg38 no pred 2 to 35.csv', index = False)

In [2]:
df = pd.read_csv('hg38 no pred 2 to 35.csv')

In [17]:
for i,r in df.iterrows():
    with open('hg38 no pred 2 to 35.fasta', 'a') as file:
        file.write('> %s | %s | %s ' % (r['Desc'].split(' ')[0], r['Acr'], r['Desc']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')
        

removing sequences sharing > 85% identity

In [36]:
pndf = pd.read_csv('pndf.csv')
p = pndf[:2862].sort_values(by = 'Len', ascending= False)
n = pndf[2862:].sort_values(by = 'Len', ascending= False)

In [22]:
pid = p[['Id', 'Len']].copy()
pid = pid.rename(columns = {'Id': 'qaccver'})

pout = pd.read_csv('pout.csv', sep ='\t', names = ['qaccver', 'saccver', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
                                             'qend', 'sstart', 'send', 'evalue', 'bitscore'])

pout = pd.merge(pout, pid, on= 'qaccver', how='left')
pout['perclen'] = pout['length']/pout['Len']
pout['finalperc'] = pout['perclen'] * (pout['pident'] / 100)
pout = pout.loc[pout['finalperc'] > 0.85]

pout = pout[['qaccver', 'saccver']].copy()

In [23]:
pout['excl'] = ''

drop = []
for i,r in pout.iterrows():
    if r['qaccver'] == r['saccver']:
        drop.append(i)
    if r['qaccver'] > r['saccver']:
        pout.loc[i,'excl'] = r['saccver'] + r['qaccver']
    else:
        pout.loc[i,'excl'] = r['qaccver'] + r['saccver']

pout = pout.drop_duplicates(subset='excl')
pout = pout.drop(index = drop)

In [49]:
hitlist = []
notexonerated = p['Id'].to_list()

for i,r in p.iterrows():
    notexonerated.remove(r['Id'])
    if r['Id'] not in hitlist:
        for i2,r2 in pout.loc[pout['qaccver'] == r['Id']].iterrows():
            if r2['saccver'] in notexonerated:
                hitlist.append(r2['saccver'])
hitlist = list(set(hitlist))

p['die'] = 'no'

ilist = []
for i,r in p.iterrows():
    if r['Id'] in hitlist:
        ilist.append(i)
        
for i in ilist:
    p.loc[i, 'die'] = 'yes'

In [66]:
p.to_csv('psk.csv', index = False)

In [67]:
nid = n[['Id', 'Len']].copy()
nid = nid.rename(columns = {'Id': 'qaccver'})
nout = pd.read_csv('nout.csv', sep ='\t', names = ['qaccver', 'saccver', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
                                             'qend', 'sstart', 'send', 'evalue', 'bitscore'])

nout = pd.merge(nout, nid, on= 'qaccver', how='left')
nout['perclen'] = nout['length']/nout['Len']
nout['finalperc'] = nout['perclen'] * ( nout['pident'] / 100 )
nout = nout.loc[nout['finalperc'] > 0.85]

nout = nout[['qaccver', 'saccver']].copy()

In [68]:
nout['excl'] = ''
drop = []
for i,r in nout.iterrows():
    if r['qaccver'] == r['saccver']:
        drop.append(i)
    if r['qaccver'] > r['saccver']:
        nout.loc[i,'excl'] = r['saccver'] + r['qaccver']
    else:
        nout.loc[i,'excl'] = r['qaccver'] + r['saccver']

nout = nout.drop_duplicates(subset='excl')
nout = nout.drop(index = drop)

In [69]:
hitlist = []
notexonerated = n['Id'].to_list()

for i,r in n.iterrows():
    notexonerated.remove(r['Id'])
    if r['Id'] not in hitlist:
        for i2,r2 in nout.loc[nout['qaccver'] == r['Id']].iterrows():
            if r2['saccver'] in notexonerated:
                hitlist.append(r2['saccver'])
hitlist = list(set(hitlist))

n['die'] = 'no'

ilist = []
for i,r in n.iterrows():
    if r['Id'] in hitlist:
        ilist.append(i)
        
for i in ilist:
    n.loc[i, 'die'] = 'yes'

In [74]:
n.to_csv('nsk.csv', index = False)

In [3]:
h1 = pd.read_csv('h1out.csv', sep ='\t', names = ['qaccver', 'saccver', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
                                             'qend', 'sstart', 'send', 'evalue', 'bitscore'])
h1 = h1[['qaccver', 'saccver', 'pident', 'length']].copy()

In [4]:
df['Id'] = [d.split(' ')[0] for d in df['Desc'].to_list()]

In [5]:
dfid = df[['Id', 'Len']].copy()

dfid = dfid.rename(columns = {'Id': 'qaccver'})

h1 = pd.merge(h1, dfid, on= 'qaccver', how='left')
h1['perclen'] = h1['length']/h1['Len']
h1['finalperc'] = h1['perclen'] * ( h1['pident'] / 100 )
h1 = h1.loc[h1['finalperc'] > 0.85]

h1 = h1[['qaccver', 'saccver']].copy()

In [6]:
h1['excl'] = ''
drop = []
for i,r in h1.iterrows():
    if r['qaccver'] == r['saccver']:
        drop.append(i)
    if r['qaccver'] > r['saccver']:
        h1.loc[i,'excl'] = r['saccver'] + r['qaccver']
    else:
        h1.loc[i,'excl'] = r['qaccver'] + r['saccver']

h1 = h1.drop_duplicates(subset='excl')
h1 = h1.drop(index = drop)

In [8]:
hitlist = []
notexonerated = df['Id'].to_list()

for i,r in df.iterrows():
    notexonerated.remove(r['Id'])
    if r['Id'] not in hitlist:
        for i2,r2 in h1.loc[h1['qaccver'] == r['Id']].iterrows():
            if r2['saccver'] in notexonerated:
                hitlist.append(r2['saccver'])
hitlist = list(set(hitlist))

df['die'] = 'no'

ilist = []
for i,r in df.iterrows():
    if r['Id'] in hitlist:
        ilist.append(i)
        
for i in ilist:
    df.loc[i, 'die'] = 'yes'

In [12]:
df[220:250]

,Desc,Acr,Seq,Len,Id,die
220,NM_001329919.2 Homo sapiens unc-13 homolog C (...,UNC13C,AGAAAAGACTCAGCCGCAGCCGGCGATGTGTGAAGTTCCCAGCACG...,13709,NM_001329919.2,no
221,NM_001080463.2 Homo sapiens dynein cytoplasmic...,DYNC2H1,ATAGCGACTACCCCTGGCAACCGCGAAGCTCTGCGGTCCCGCGGTC...,13704,NM_001080463.2,no
222,NM_001009899.4 Homo sapiens upstream transcrip...,USF3,GGGAAAGTTGAGGCCGCGCTGCCGCCGGGTCTCCGGCCTTGAGGGC...,13704,NM_001009899.4,no
223,NM_001281956.2 Homo sapiens CUB and Sushi mult...,CSMD2,ATTCCAGCCGCAGAAGCTCAGGGCTCCCGCGTACCGGGATTTTTTC...,13655,NM_001281956.2,no
224,NM_018012.4 Homo sapiens kinesin family member...,KIF26B,AGTTACCAAGCTCGGTGAAGGAGACAAGTTCCCACAGCTGACTCGG...,13593,NM_018012.4,no
225,NM_022055.2 Homo sapiens potassium two pore do...,KCNK12,AGTGACTCCGCCTCTCCCTGCCGGGCGGCTCTTCGGCTGGAGCTGA...,13564,NM_022055.2,no
226,NM_001080436.2 Homo sapiens WT1 interacting pr...,WTIP,CATTCCTCCGCCGCGCGCGCCGCCGAGCAGGGCGCCGCGTCCCCCG...,13545,NM_001080436.2,no
227,NM_001330450.2 Homo sapiens mitogen-activated ...,MAP2K6,AGTTCTGTTTCTCCTTGCCGAAGTGTGGTCTTTGGAGCTAAGTGAA...,13525,NM_001330450.2,no
228,"NM_001277313.2 Homo sapiens formin 1 (FMN1), t...",FMN1,CCTTTGGGCTTTGGCTCTGGACTGGAGCGCAGCATCCTTCGAGGCT...,13511,NM_001277313.2,no
229,NM_152592.5 Homo sapiens spectrin repeat conta...,SYNE3,CCCCTGCAGGTGCCATGACTCAGCAGCCCCAGGACGACTTTGACAG...,13502,NM_152592.5,no


In [9]:
df.loc[df['die'] == 'yes']

,Desc,Acr,Seq,Len,Id,die
242,"NR_046235.3 Homo sapiens RNA, 45S pre-ribosoma...",RNA45SN5,GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCC...,13357,NR_046235.3,yes
244,"NR_145819.1 Homo sapiens RNA, 45S pre-ribosoma...",RNA45SN1,GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCC...,13351,NR_145819.1,yes
248,"NR_146144.1 Homo sapiens RNA, 45S pre-ribosoma...",RNA45SN2,GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCC...,13315,NR_146144.1,yes
249,"NR_146151.1 Homo sapiens RNA, 45S pre-ribosoma...",RNA45SN3,GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCC...,13309,NR_146151.1,yes
1423,NM_001348050.2 Homo sapiens ERV3-1-ZNF117 read...,ERV3-1-ZNF117,GTCCAATCAGGCACTCAGCTGGAGCAGACAGGACGGCTTCCGGGTT...,8196,NM_001348050.2,yes
...,...,...,...,...,...,...
15389,NR_028084.1 Homo sapiens family with sequence ...,FAM41AY2,ATAAAACCCATCTAAAATGACCTTCTGGGCTACTGACTGCTCACTA...,2043,NR_028084.1,yes
15469,NM_001130167.2 Homo sapiens pregnancy specific...,PSG8,AGAAGGAGGCAGGACAGCACTGCTGAGAGCTGTGCTCAGGAAGCTT...,2031,NM_001130167.2,yes
15499,NM_001321400.1 Homo sapiens MAGE family member...,MAGEA2B,TTGCGCATTGGAGGTCAGAGGACAGCGAGATTCTCGCCCTGAGCAA...,2026,NM_001321400.1,yes
15511,NR_026595.2 Homo sapiens family with sequence ...,FAM226A,GGGCGGCGGTCTCCGACTCAAGACCCTGGGGCTCCGGGTCTTCTTA...,2023,NR_026595.2,yes


In [13]:
df.to_csv('h1sk.csv', index = False)